In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from biokb.llm import AIRLLM
from biokb.utils import get_logger
from biokb.helpers import get_generation_config
from biokb.tools import search_tool, response
from biokb.settings import MODEL_NAME, DEVICE_NAME, MAX_TIME

2024-10-13 10:20:02,297 - air-llm - INFO - Starting AIR llm...
/users/gast00/prashant/air-rag/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [39]:
MODEL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# MODEL_NAME = "mistralai/Mixtral-8x7B-v0.1"

In [40]:
logger = get_logger(debug=True)
logger.info("Starting the AIR Agent")

2024-10-13 10:45:06,767 - air-rag - INFO - Starting the AIR Agent


In [41]:
generation_config = get_generation_config()
llm = AIRLLM(
    model_name=MODEL_NAME,
    generation_config=generation_config,
    logger=logger
)

/users/gast00/prashant/air-rag/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
2024-10-13 10:45:08,534 - air-rag - INFO - Loading model: meta-llama/Meta-Llama-3.1-8B-Instruct
2024-10-13 10:45:08,536 - air-rag - INFO - Loading model in 4-bit mode
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2024-10-13 10:45:14,352 - air-rag - INFO - Model loaded: meta-llama/Meta-Llama-3.1-8B-Instruct


## Tokenizer

### Eg 1

In [242]:
messages = [
    #{"role": "system", "content": "you are a helpful chat assistant with access to a large database of scientific papers.  Only use search tool when you cannot answer based on this input. Try your best to answer user queries based on the input."},
    # # {"role": "assistant", "tool_call": {"name": "get_weather", "parameters": {"location": "California"}}},
    # {"role": "assistant", "content": "23 degrees celsius"},
    {"role": "system", "content": "You are a chat agent. Use 'response' tool to generate responses. Use 'search' tool to search before you asnwer any scientific questions. Do not give user any scientific answer without using search tool."},
    {"role": "user", "content": "hi"},
    {"role": "assistant", "content": "Hello! How can I help you today?"},
    {"role": "user", "content": "What is p53"},
    {"role": "assistant", "content": {"tool_call": {"name": "search", "parameters": {"query": "p53"}}}},
    {"role": "assistant", "content": "p53 is a protein that in humans is encoded by the TP53 gene. p53 is crucial in multicellular organisms, where it regulates the cell cycle and, thus, functions as a tumor suppressor that is involved in preventing cancer."},
    {"role": "user", "content": "Ok"},
    {"role": "assistant", "content": "That's a good start! Would you like to know more about the p53 protein and its role in cancer prevention?"},
    {"role": "user", "content": "Yes"},
    {"role": "assistant", "content": {"tool_call": {"name": "search", "parameters": {"query": "p53 and cancer"}}}},
    {"role": "assistant", "content": "The p53 pathway plays a crucial role in regulating the biological behavior and determining the fate of tumor cells in non-small cell lung cancers (NSCLCs). Mutations of p53, a reduced p14 alternate reading frame expression, a reduced herpesvirus-associated ubiquitin-specific protease expression, or a reduced p33 inhibitor of growth gene1b expression are common in NSCLCs. The balance of expression of p53 target genes, such as p21, Bax, and PUMA, regulates the biological behavior and determines the fate of tumor cells. Many studies on cancer gene therapy using these molecules associated with the p53 pathway have been performed to develop new strategies for treating NSCLC patients. Therefore, the establishment of a comprehensive and simple evaluation protocol for the p53 pathway is required for clinical use. Additionally, the p53 tumor suppressor gene has been shown to regulate expression of some metastasis-related genes, including epidermal growth factor receptor, matrix metalloproteinase (MMP)-2, cathepsin D, and thrombospondin-1, but represses expression of the genes encoding basic fibroblast growth factor and multidrug resistance-1. Decreased expression of E-cadherin is associated with p53 alterations. The p53 pathway also regulates the expression of the metastasis suppressor gene Nm23-H1, which encodes an 18-kDa nucleoside diphosphate kinase. Wild-type p53 upregulates the expression of Nm23-H1 at protein and mRNA levels in MCF-7 and J7B cells, while mutant p53 downregulates the expression of Nm23-H1 in RKO and H1299 cells"},
    {"role": "user", "content": "Ok"},
    {"role": "assistant", "content": "Would you like to know more about the p53 pathway and its role in cancer prevention?"},
    {"role": "user", "content": "NO"},
]

In [244]:
# Tokenize the input messages
output = llm.tokenizer.apply_chat_template(
    conversation = messages,
    return_tensors="pt", 
    tools = [response, search_tool],
    tokenize=True, 
    add_generation_prompt=False,
    return_dict=True,

)
input_ids = output["input_ids"].to(DEVICE_NAME)
attn_mask = output["attention_mask"].to(DEVICE_NAME)


In [245]:
# Input message for the model
print(llm.tokenizer.decode(input_ids[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a chat agent. Use'response' tool to generate responses. Use'search' tool to search before you asnwer any scientific questions. Do not give user any scientific answer without using search tool.<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.Do not use variables.

{
    "type": "function",
    "function": {
        "name": "response",
        "description": "A tool to respond to the user message.",
        "parameters": {
            "type": "object",
            "properties": {
                "response": {
                    "type": "string",
                    "description": "The response 

In [246]:
output = llm.model.generate(
    input_ids = input_ids,
    attention_mask = attn_mask,
    max_length=15000, 
    max_time=MAX_TIME,
    generation_config=get_generation_config(),
    repetition_penalty=1.5,
    do_sample=False,    
)
output = llm.tokenizer.decode(output[0], skip_special_tokens=False)

/users/gast00/prashant/air-rag/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/users/gast00/prashant/air-rag/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [247]:
print(output)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a chat agent. Use'response' tool to generate responses. Use'search' tool to search before you asnwer any scientific questions. Do not give user any scientific answer without using search tool.<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.Do not use variables.

{
    "type": "function",
    "function": {
        "name": "response",
        "description": "A tool to respond to the user message.",
        "parameters": {
            "type": "object",
            "properties": {
                "response": {
                    "type": "string",
                    "description": "The response 

: 

### Eg 2

In [162]:
messages = [
    {'role': 'system',
    'content': 'you are a helpful chat assistant with access to a large database of scientific papers. You may search tool to gather information about user query domain and answer user queries based on what you learned.'},
    {'role': 'user', 'content': 'Hi'},
    {'role': 'assistant', 'content': 'Hi, how can I assist you?'},
    {'role': 'user', 'content': 'What is p53?'},
    {'role': 'assistant', 'content': 'p53 is a protein that in humans is encoded by the TP53 gene. p53 is crucial in multicellular organisms, where it regulates the cell cycle and functions as a tumor suppressor. p53 has been described as "the guardian of the genome" because of its role in conserving stability by preventing genome mutation.'},
    {'role': 'user', 'content': 'name the genes'}
]

In [163]:
# Tokenize the input messages
output = llm.tokenizer.apply_chat_template(
    conversation = messages,
    tools = [search_tool, response],
    return_tensors="pt", 
    tokenize=True, 
    add_generation_prompt=True,
    return_dict=True
)
input_ids = output["input_ids"].to(DEVICE_NAME)
attn_mask = output["attention_mask"].to(DEVICE_NAME)


In [164]:
# Input message for the model
print(llm.tokenizer.decode(input_ids[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

you are a helpful chat assistant with access to a large database of scientific papers. Use search tool to gather information about user query domain and answer user queries based on what you learned.<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.Do not use variables.

{
    "type": "function",
    "function": {
        "name": "search_tool",
        "description": "Use this tool to search for information in the database.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description

In [165]:
output = llm.model.generate(
    input_ids = input_ids,
    attention_mask = attn_mask,
    max_length=15000, 
    max_time=MAX_TIME,
    generation_config=get_generation_config(),
    repetition_penalty=1.5,
    do_sample=False,    
)
output = llm.tokenizer.decode(output[0], skip_special_tokens=False)
print(output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

you are a helpful chat assistant with access to a large database of scientific papers. Use search tool to gather information about user query domain and answer user queries based on what you learned.<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.Do not use variables.

{
    "type": "function",
    "function": {
        "name": "search_tool",
        "description": "Use this tool to search for information in the database.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description

### Eg 3

In [28]:
messages = [{'role': 'system', 'content': 'You are a knowledgeable chat assistant.'},
  #Use `response` tool to respond to user. You may use `search_tool` to gather information related to biology and proteins.'},
 {'role': 'user', 'content': 'Hi'},
 {'role': 'assistant', 'content': 'Hi, how can I assist you?'},
 {'role': 'user', 'content': 'what is p53'},
# {"role": "assistant", "tool_call": {"type": "function", "id": "ASDVSDVG", "function": {"name": "search_tool", "parameters": {"query": "p53"}}}},
# {'role': 'tool', 'tool_call_id': "ASDVSDVG", 'name': 'search_tool', 
{'role': 'assistant',
'content': 'p53, also known as Tumor protein P53, cellular tumor antigen p53 (UniProt name), or transformation-related protein 53 (TRP53) is a regulatory protein that is often mutated in human cancers. The p53 proteins (originally thought to be, and often spoken of as, a single protein) are crucial in vertebrates, where they prevent cancer formation.'},
# {'role': 'assistant', 'content': 'p53, also known as Tumor protein P53, cellular tumor antigen p53 (UniProt name), or transformation-related protein 53 (TRP53) is a regulatory protein that is often mutated in human cancers. The p53 proteins (originally thought to be, and often spoken of as, a single protein) are crucial in vertebrates, where they prevent cancer formation.'},
# {'role': 'user', 'content': 'ok'},
]

In [30]:
def output_tool(type: str, parameters: str):
    """AIRLLM output tool
    Args:
        type: The response type of the model, either search to call or response to user.
        parameters: The parameters of the response or search
    """
    return {"role": response, "parameters": parameters}

In [31]:
# Tokenize the input messages
output = llm.tokenizer.apply_chat_template(
    conversation = messages,
    #tools = [search_tool, response],
    tools = [output_tool],
    return_tensors="pt", 
    tokenize=True, 
    add_generation_prompt=True,
    return_dict=True
)
input_ids = output["input_ids"].to(DEVICE_NAME)
attn_mask = output["attention_mask"].to(DEVICE_NAME)


In [32]:
print(llm.tokenizer.decode(input_ids[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a knowledgeable chat assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.Do not use variables.

{
    "type": "function",
    "function": {
        "name": "output_tool",
        "description": "AIRLLM output tool",
        "parameters": {
            "type": "object",
            "properties": {
                "type": {
                    "type": "string",
                    "description": "The response type of the model, either search to call or response to user."
                },
                "parameters": {
                    "type": "string",
                    "descripti

In [34]:
output = llm.model.generate(
    input_ids = input_ids,
    attention_mask = attn_mask,
    max_length=15000, 
    max_time=MAX_TIME,
    generation_config=get_generation_config(),
    repetition_penalty=1.5,
    do_sample=False,    
)
output = llm.tokenizer.decode(output[0], skip_special_tokens=False)
print(output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a knowledgeable chat assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.Do not use variables.

{
    "type": "function",
    "function": {
        "name": "output_tool",
        "description": "AIRLLM output tool",
        "parameters": {
            "type": "object",
            "properties": {
                "type": {
                    "type": "string",
                    "description": "The response type of the model, either search to call or response to user."
                },
                "parameters": {
                    "type": "string",
                    "descripti